In [139]:
import time

import tensorflow as tf
import numpy as np

from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib

from keras.datasets import cifar10, mnist

# Write TFRecord

In [203]:
# Modified code from TensorFlow
# Reference: tensorflow/examples/how_tos/reading_data/convert_to_records.py
def _int64_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to(data_set, name):
    """Converts a dataset to tfrecords."""
    images = data_set[0]
    labels = data_set[1]
    num_examples = data_set[2]

    if images.shape[0] != num_examples:
        raise ValueError('Images size %d does not match label size %d.' %
                     (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    if len(images.shape == 4):
        depth = images.shape[3]
    else:
        depth = 1

    filename =  name + '.tfrecords'
    print('Writing', filename, end=' ')
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()
    print('done.')


In [204]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [206]:
def oneHotEncoding(labels):
    numberOfClasses = len(list(set(labels)))
    labelsOneHotEncodig = np.zeros([len(labels), numberOfClasses])
    for i, label in enumerate(labels):
        labelsOneHotEncodig[i][label] = 1
    return labelsOneHotEncodig
type(oneHotEncoding(y_train))

numpy.ndarray

In [208]:
convert_to([x_train, y_train, len(x_train)], 'train')
#convert_to([x_test, y_test, len(x_test)], 'test')

Writing train.tfrecords done.


# Read TFRecord

In [223]:
# Modified code from TensorFlow
# Reference: tensorflow/examples/how_tos/reading_data/fully_connected_reader.py

def read_and_decode(filename_queue, shape):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={'image_raw': tf.FixedLenFeature([], tf.string),
                                                 'label': tf.FixedLenFeature([], tf.float32),})

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.reshape(image, shape)
    #image.set_shape([image_pixels])

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(image, tf.float32)# * (1. / 255) - 0.5

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)
    #label = tf.reshape(image, [1])
    return image, label


def inputs(train, batch_size, num_epochs, filename, shape):
    if not num_epochs: num_epochs = None

    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
        [filename], num_epochs=num_epochs)

    # Even when reading in multiple threads, share the filename
    # queue.
    image, label = read_and_decode(filename_queue, shape)

    # Shuffle the examples and collect them into batch_size batches.
    # (Internally uses a RandomShuffleQueue.)
    # We run this in two threads to avoid being a bottleneck.
    images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=1000 + 3 * batch_size,
        # Ensures a minimum amount of shuffling of examples.
        min_after_dequeue=1000)
    #sparse_labels = tf.reshape(sparse_labels, [-1, 10])
    return images, sparse_labels

In [224]:
images, labels = inputs(train=True, batch_size=128,  num_epochs=5, 
                        filename='./train.tfrecords', shape=[32, 32, 3])

In [225]:
images

<tf.Tensor 'shuffle_batch_4:0' shape=(128, 32, 32, 3) dtype=float32>

In [226]:
labels

<tf.Tensor 'shuffle_batch_4:1' shape=(128, 10) dtype=float32>

# Build the model

In [227]:
def alex_net(inputs, n_classes, prob=1., verbose=False):
    input_layer = tf.reshape(inputs, [-1, 28, 28, 1])
    print(input_layer) if verbose else 0
    net = layers.conv2d(input_layer, 64, [3, 3], 2, padding='SAME', scope='conv1', activation_fn=tf.nn.relu, \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool1')
    print(net) if verbose else 0
    net = layers.conv2d(net, 192, [3, 3], scope='conv2', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 1, scope='pool2')
    print(net) if verbose else 0
    net = layers.conv2d(net, 384, [3, 3], scope='conv3', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.conv2d(net, 384, [3, 3], scope='conv4', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.conv2d(net, 256, [3, 3], scope='conv5', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool5')
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=4096, scope='fc1', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.dropout(net, prob)
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=4096, scope='fc2', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.dropout(net, prob)
    print(net) if verbose else 0
    net = layers.fully_connected(net, num_outputs=n_classes, scope='fc3', activation_fn=tf.nn.relu,  \
                        weights_initializer=tf.truncated_normal_initializer(0, 0.1), \
                        biases_initializer=tf.truncated_normal_initializer(0.3, 0.1))
    print(net) if verbose else 0
    net = layers.softmax(net)
    print(net) if verbose else 0
    return net

In [232]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def LeNet5_Model(data, keep_prob=.5, activation='relu'):    
    # --- level 1 --- #
    #A 4-D tensor of shape [filter_height, filter_width, in_channels, out_channels]
    # in_channels depens on the input img, out_channels is the number of desired filters
    filters_conv_1 = weight_variable([5, 5, 1, 6])
    window_pool_1 = [1, 2, 2, 1]
    strides_conv_1 = [1, 1, 1, 1]
    strides_pool_1 = [1, 2, 2, 1]
    #the bias term is applied to each feature map
    bias_layer_1 = bias_variable([6])
    #we use SAME as padding because we want to keep the size of the original img 28x28,
    #NOT 32x32 as expected
    conv_layer_1 = tf.nn.conv2d(data, filters_conv_1, strides_conv_1, "SAME")
    if activation=='relu':
        hidden_layer_1 = tf.nn.relu(conv_layer_1 + bias_layer_1)
    else:
        hidden_layer_1 = tf.nn.sigmoid(conv_layer_1 + bias_layer_1)
    pool_layer_1 = tf.nn.max_pool(hidden_layer_1, window_pool_1, strides_pool_1, "VALID")
    
    # --- end level 1 --- #
    # --- level 2 --- #
    filters_conv_2 = weight_variable([5, 5, 6, 16])
    bias_layer_2 = bias_variable([16])
    strides_conv_2 = [1, 1, 1, 1]
    strides_pool_2 = [1, 2, 2, 1]
    window_pool_2 = [1, 2, 2, 1]
    strides_pool_2 = [1, 2, 2, 1]
    
    conv_layer_2 = tf.nn.conv2d(pool_layer_1, filters_conv_2, strides_conv_2, "VALID")
    if activation=='relu':
        hidden_layer_2 = tf.nn.relu(conv_layer_2 + bias_layer_2)
    else:
        hidden_layer_2 = tf.nn.sigmoid(conv_layer_2 + bias_layer_2)
    pool_layer_2 = tf.nn.max_pool(hidden_layer_2, window_pool_2, strides_pool_2, "VALID")
    flatten_l2 = tf.contrib.layers.flatten(pool_layer_2)
    # --- end level 2 --- #

    # --- level 3 --- #
    l3_weights = weight_variable([16*5*5, 120])
    l3_bias = bias_variable([120])
    if activation=='relu':
        hidden_layer_3 = tf.nn.relu(tf.matmul(flatten_l2, l3_weights) + l3_bias)
    else:
        hidden_layer_3 = tf.nn.sigmoid(tf.matmul(flatten_l2, l3_weights) + l3_bias)     
    # --- end level 3 --- #
    
    # --- dropout layer --- #
    drop_layer_1 = tf.nn.dropout(hidden_layer_3, keep_prob)
    # --- end dropout layer --- #
    
    # --- level 4 --- #
    l4_weights = weight_variable([120,84])
    l4_bias = bias_variable([84])
    if activation=='relu':
        hidden_layer_4 = tf.nn.relu(tf.matmul(drop_layer_1, l4_weights) + l4_bias) 
    else:
        hidden_layer_4 = tf.nn.sigmoid(tf.matmul(drop_layer_1, l4_weights) + l4_bias) 
    # --- end level 4 --- #
    
    # --- dropout layer --- #
    drop_layer_2 = tf.nn.dropout(hidden_layer_4, keep_prob)
    # --- end dropout layer --- #
    
    # --- level 5 --- #
    l5_weights = weight_variable([84, 10])
    l5_bias = bias_variable([10])
    output = tf.nn.softmax(tf.matmul(drop_layer_2, l5_weights) + l5_bias)
    # output = tf.matmul(hidden_layer_4, l5_weights) + l5_bias
    # --- end level 5 --- #
    return output

In [233]:
def train(learning_rate=0.01, training_epochs=1, batch_size=128, \
          display_step=1, logs_path='./logs', keep_probability= 1.0):

    tf.reset_default_graph()
    
    x, y = inputs(train=True, batch_size=batch_size, num_epochs=training_epochs, \
                                filename='./train.tfrecords', shape=[28, 28, 1])
    
    #pred = alex_net(x, 10, keep_probability)
    pred = LeNet5_Model(x)
    
    loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred+1e-9), reduction_indices=1))
    #tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.argmax(pred, axis=1), labels=(y)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), tf.float32))

    init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

    tf.summary.scalar("Loss", loss)
    tf.summary.scalar("Accuracy", accuracy)
    merged_summary_op = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        session.run(init)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=session, coord=coord)
        
        step = 0
        try: 
            while not coord.should_stop():
                start_time = time.time()
                _, loss_value, summary = session.run([optimizer, loss, merged_summary_op])
                duration = time.time() - start_time

                # Print an overview fairly often.
                if step % display_step == 0:
                    start_time = time.time()
                    print('Step %d: loss = %.2f - accuracy = %.2f (%.3f sec)' % (step, loss_value, accuracy.eval(), duration))
                summary_writer.add_summary(summary, step)
                step += 1  
        except Exception:
            print('Done training for %d epochs, %d steps.' % (num_epochs, step))
        finally:
            coord.request_stop()
    coord.join(threads)
    sess.close()

In [ ]:
train(batch_size=100, display_step=100, training_epochs=100)

Step 0: loss = 16.55 - accuracy = 0.07 (0.219 sec)
Step 100: loss = 0.89 - accuracy = 0.80 (0.052 sec)
Step 200: loss = 0.40 - accuracy = 0.88 (0.053 sec)
Step 300: loss = 0.45 - accuracy = 0.91 (0.055 sec)
Step 400: loss = 0.15 - accuracy = 0.86 (0.055 sec)
Step 500: loss = 0.22 - accuracy = 0.91 (0.054 sec)
Step 600: loss = 0.48 - accuracy = 0.86 (0.057 sec)
Step 700: loss = 0.25 - accuracy = 0.92 (0.053 sec)
Step 800: loss = 0.26 - accuracy = 0.92 (0.051 sec)
Step 900: loss = 0.08 - accuracy = 0.94 (0.056 sec)
Step 1000: loss = 0.32 - accuracy = 0.93 (0.057 sec)
